# Build test

* Connect to remote host

In [49]:
import os
from paramiko import SSHClient
from scp import SCPClient

USER='hadoop'
HOST='172.16.4.135'
PASSWD='hadoop'
REMOTE_WORKING_DIR = 'bloom'
REMOTE_PARAMS=f'/home/{USER}/{REMOTE_WORKING_DIR}/params'
REMOTE_LOGS=f'/home/{USER}/{REMOTE_WORKING_DIR}/logs'
PARAMS='data/params'
LOGS='data/logs'

client = SSHClient()
client.load_system_host_keys()
client.connect(hostname=HOST, username=USER, password=PASSWD)

* Upload compute params script

In [ ]:
SCRIPTS = ['scripts/compute_params.sh']
scp = SCPClient(client.get_transport())
scp.put(SCRIPTS, remote_path=REMOTE_WORKING_DIR)
scp.close()

* Download params from cluster 

In [ ]:
os.makedirs('data/', exist_ok=True)
scp = SCPClient(client.get_transport())
scp.get(REMOTE_PARAMS, recursive=True, local_path='data/')
scp.get(REMOTE_LOGS, recursive=True, local_path='data/')
scp.close()

* Create input files (M,K,#Mapper)

In [ ]:
import os
import csv
from math import ceil

folders = os.walk(PARAMS)
os.makedirs('scripts/input/mappers', exist_ok=True)

INPUT='scripts/input'

with open('data/title.ratings.tsv') as f_in: 
    N = num_lines = sum(1 for line in f_in) - 1

DEFAULT_MAPPERS = ceil(N / 8)
n_mappers = [4, 6, 8, 10, 12] # 2, 1.33, 1, 0.8, 0.66 map/core
n_splits = [ceil(N/m) for m in n_mappers]

next(folders, None) # skip first element
for folder in folders:  

    tokens=folder[0].replace('\\','/').split('/')[-1].split('_')

    P=float(tokens[1][1:])
    K=int(tokens[2][1:])
    if K == 1:
        continue
    params_file = f'{folder[0]}/{folder[2][0]}'

    with(open(params_file)) as params:   
        reader = csv.reader(params, delimiter="\t")
        input = f'echo "title.ratings.tsv $1 {DEFAULT_MAPPERS} '
        input_filename = f'{tokens[1]}{tokens[2]}'
        m_size = ''
        k=0
        with open(f'{INPUT}/{input_filename}', 'w') as input_file:
            for line in reader:
                m_size += f'{line[3]} '
                k=line[4]
            input += f'{m_size}{k} $2"'
            input_file.write(input)
        if K == 0:
            for n_mapper, n_split in zip(n_mappers, n_splits):
                with open(f'{INPUT}/{input_filename}MAP{n_mapper}', 'w') as input_mapper_file:
                    input_mapper = f'echo "title.ratings.tsv $1 {n_split} {m_size}{k} $2"'
                    input_mapper_file.write(input_mapper)

* Upload test and input

In [50]:

TEST_DIR = 'scripts/input'
SCRIPTS = ['scripts/test.sh', 'scripts/clean.sh']
scp = SCPClient(client.get_transport())

scp.put(TEST_DIR, recursive=True, remote_path=REMOTE_WORKING_DIR)
scp.put(SCRIPTS, remote_path=REMOTE_WORKING_DIR)

scp.close()
client.close()